# Phase 1: Environment & Setup
We are building a production RAG system. We start in a notebook to **prototype components** before moving them to Python modules.

### Step 1: Load Environment Variables
Since our `.env` is in a sibling directory, we need to load it explicitly.

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

# 1. Define path to the .env file in the sibling directory
# Adjust 'RAG systems using LlamaIndex' if your folder name is different
env_path = Path("..") / "RAG systems using LlamaIndex" / ".env"

# 2. Load it
if env_path.exists():
    load_dotenv(dotenv_path=env_path)
    print("✅ Loaded .env file")
else:
    print(f"❌ Could not find .env at: {env_path.resolve()}")

# 3. Verify
if os.getenv("GOOGLE_API_KEY"):
    print("✅ API Key found")
else:
    print("⚠️ API Key not found. Check your .env file.")

✅ Loaded .env file
✅ API Key found


### Step 2: The Ingestion Logic
**Design Decision**: Clone **Everything**, Filter **Later**.
*   **Cloning**: Use `git clone`. It's faster and simpler to just pull the whole repo than to try and "sparse checkout" file-by-file.
*   **Filtering**: We will apply a **Blacklist** (exclude `.png`, `.pth`, `.git`) during the document loading phase. This keeps the local disk faithful to reality but protects the embedding model from garbage.

In [2]:
import subprocess
import shutil

# CONFIG
REPO_URL = "https://github.com/XunpengYi/Diff-Retinex-Plus"
STORAGE_PATH = Path("./github_rag_engine/data/repos")
REPO_NAME = REPO_URL.split("/")[-1]
LOCAL_REPO_PATH = STORAGE_PATH / REPO_NAME

def clone_repo(url, path):
    if path.exists():
        print(f"ℹ️ Repo already exists at: {path}")
        # In production, we would git pull here
        return
    
    print(f"⏳ Cloning {url}...")
    try:
        subprocess.run(["git", "clone", url, str(path)], check=True)
        print("✅ Clone successful")
    except subprocess.CalledProcessError as e:
        print(f"❌ Clone failed: {e}")

# 1. Run the clone
clone_repo(REPO_URL, LOCAL_REPO_PATH)

ℹ️ Repo already exists at: github_rag_engine/data/repos/Diff-Retinex-Plus


### Step 3: Filtering & Processing
Now we iterate through the files. 

**The Rule**: skip if extension is in `IGNORED_EXTENSIONS` OR if it is a hidden file/directory (starts with `.`).

In [3]:
IGNORED_EXTENSIONS = {
    # Images
    ".png", ".jpg", ".jpeg", ".gif", ".svg", ".ico",
    # Compiled / Binary
    ".pyc", ".o", ".exe", ".dll", ".so", ".dylib",
    # Models / Data
    ".pth", ".pt", ".onnx", ".pkl", ".bin", ".data",
    # Git
    ".git",
    # Lock files (Optional - debating this)
    ".lock"
}


In [4]:

def get_source_files(directory):
    source_files = []
    for root, dirs, files in os.walk(directory):
        # 1. Remove hidden dirs like .git in-place to avoid traversing them
        dirs[:] = [d for d in dirs if not d.startswith(".")]
        
        for file in files:
            # 2. Skip hidden files
            if file.startswith("."):
                continue
                
            # 3. Check extension blacklist
            ext = os.path.splitext(file)[1].lower()
            if ext in IGNORED_EXTENSIONS:
                continue
            
            full_path = Path(root) / file
            source_files.append(full_path)
            
    return source_files

In [5]:

# Test the filter
files = get_source_files(LOCAL_REPO_PATH)
print(f"Found {len(files)} valid source files")
print("Sample:", [f.name for f in files[:5]])

Found 44 valid source files
Sample: ['test.py', 'README.md', 'test_UHD.py', 'train.py', 'environment.yaml']


### Step 4: The "Repo Map" (The "What was here" context)
You made a great point: if we skip `.pth` files, the LLM won't know they exist. 

**The Solution**: We create a single special document called the **Repo Map**. 
This is a text file containing the tree structure of the *entire* repo (including the files we ignored). We index this first. This gives the application "Spatial Awareness".

In [6]:
def generate_repo_map(root_dir):
    lines = []
    root_path = Path(root_dir)

    for root, dirs, files in os.walk(root_path):
        if ".git" in dirs:
            dirs.remove(".git")

        for f in files:
            rel_path = Path(root, f).relative_to(root_path)
            ext = rel_path.suffix.lower()

            marker = ""
            if f.startswith(".") or ext in IGNORED_EXTENSIONS:
                marker = " [IGN]"

            lines.append(f"{rel_path.as_posix()}{marker}")

    return "\n".join(sorted(lines))

In [7]:
# Generate and Print
repo_map_str = generate_repo_map(LOCAL_REPO_PATH)
print("--- REPO MAP PREVIEW ---")
print(repo_map_str)

--- REPO MAP PREVIEW ---
README.md
archs/DRNet_Retinex_arch.py
archs/MoE_arch.py
archs/__init__.py
archs/ddpm_arch.py
archs/decom_arch.py
archs/sr3unet_arch.py
asserts/framework.png [IGN]
data/__init__.py
data/lol_dataset.py
dataset/put_the_dataset_here.md
decom_tools/decom_arch.py
decom_tools/loss_decom_TDN.py
decom_tools/my_dataset.py
decom_tools/test_decom_checkpoint.py
decom_tools/train_decom.py
decom_tools/transforms.py
decom_tools/utils.py
decom_weights/put_the_decom_weights_here.md
environment.yaml
lolv1_offical_results/1.png [IGN]
lolv1_offical_results/111.png [IGN]
lolv1_offical_results/146.png [IGN]
lolv1_offical_results/179.png [IGN]
lolv1_offical_results/22.png [IGN]
lolv1_offical_results/23.png [IGN]
lolv1_offical_results/493.png [IGN]
lolv1_offical_results/547.png [IGN]
lolv1_offical_results/55.png [IGN]
lolv1_offical_results/665.png [IGN]
lolv1_offical_results/669.png [IGN]
lolv1_offical_results/748.png [IGN]
lolv1_offical_results/778.png [IGN]
lolv1_offical_results/780.

### Step 5: Loading with Metadata
Now we load the *content* of the valid files. 

**Crucial**: We inject `file_metadata`. Every chunk must know its own filename.

In [8]:
from llama_index.core import SimpleDirectoryReader, Document


In [9]:
# 1. Create the Repo Map Document
map_doc = Document(
    text=repo_map_str,
    metadata={"file_path": "REPO_STRUCTURE.txt", "type": "structure"}
)

In [10]:
import datetime

# 2. Helper to extract metadata from files
def file_metadata_extractor(file_path):
    # Get modification time
    #stat = os.stat(file_path)
    #timestamp = stat.st_mtime
    # Format as readable string
    #last_mod = datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")

    return {
        "file_path": file_path,
        "file_name": os.path.basename(file_path),
        "extension": os.path.splitext(file_path)[1],
        "repo_path": os.path.dirname(file_path),
        #"last_modified": last_mod
    }

In [11]:
if not files:
    print("⚠️ No files found to index!")
    documents = []
else:
    reader = SimpleDirectoryReader(
        input_files=[str(f) for f in files],
        file_metadata=file_metadata_extractor
    )
    code_documents = reader.load_data()
    all_documents = [map_doc] + code_documents
    print(f"✅ Loaded {len(code_documents)} code documents + 1 Repo Map")

✅ Loaded 44 code documents + 1 Repo Map


In [12]:
print(code_documents[0].metadata)

{'file_path': 'github_rag_engine/data/repos/Diff-Retinex-Plus/test.py', 'file_name': 'test.py', 'extension': '.py', 'repo_path': 'github_rag_engine/data/repos/Diff-Retinex-Plus'}


### Step 6: Settings (OpenAI)
We use **OpenAI** for both embedding and LLM.

In [13]:
# Imports
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

In [14]:
def configure_llm_settings():
    Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
    Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)
    print("✅ Configured: text-embedding-3-small & GPT-4o-mini")

In [15]:
configure_llm_settings()

✅ Configured: text-embedding-3-small & GPT-4o-mini


### Step 7: Adaptive Chunking (File-Type Aware & Tiktoken)
You are absolutely right. `.py` is handled by `CodeSplitter`, but `.md` needs `MarkdownNodeParser`, and `.yaml` needs standard text splitting.

**Refinement**: We use **tiktoken** to ensure our chunks are measured in *tokens*, not characters. This is critical for maximizing the LLM's context window usage.

In [16]:
import tiktoken
from llama_index.core.node_parser import CodeSplitter, MarkdownNodeParser, SentenceSplitter

In [17]:
def get_nodes_adaptive(docs):
    alln = []
    tok = tiktoken.encoding_for_model("gpt-4o-mini").encode

    ps = CodeSplitter(
        language="python",
        chunk_lines=150,
        chunk_lines_overlap=30,
        max_chars=3000
    )

    ms = MarkdownNodeParser(
        chunk_size=800,
        chunk_overlap=100
    )

    ts = SentenceSplitter(
        chunk_size=512,
        chunk_overlap=50,
        tokenizer=tok
    )

    for doc in docs:
        fn = doc.metadata.get("file_name", "")
        ext = doc.metadata.get("extension", "").lower()

        if ext == ".py":
            nds = ps.get_nodes_from_documents([doc])
        elif ext == ".md":
            nds = ms.get_nodes_from_documents([doc])
        else:
            nds = ts.get_nodes_from_documents([doc])

        for i, n in enumerate(nds):
            n.metadata["chunk_id"] = i
            n.metadata["file_ext"] = ext
            n.metadata["file_name"] = fn

            pre = nds[i - 1].text if i > 0 else ""
            post = nds[i + 1].text if i < len(nds) - 1 else ""

            n.metadata["pre_context"] = pre
            n.metadata["post_context"] = post

        alln.extend(nds)

    return alln

In [18]:
def inspect_nodes(nodes, sample_exts=(".py", ".md", ".yaml", ".yml")):
    seen = set()
    for n in nodes:
        ext = n.metadata.get("file_ext", "")
        if ext in sample_exts and ext not in seen:
            seen.add(ext)
            print("\n--- Sample Chunk for", ext, n.metadata.get("file_name", ""))
            print("[Node Type: {}]".format(n.class_name()))
            print("chunk_id:", n.metadata.get("chunk_id"))
            txt = n.text[:400].replace("\n", "\\n")
            print(txt + "...")
            pre = n.metadata.get("pre_context", "")
            post = n.metadata.get("post_context", "")
            print("\npre_context (first 200 chars):")
            print(pre[:200].replace("\n", "\\n"))
            print("\npost_context (first 200 chars):")
            print(post[:200].replace("\n", "\\n"))



In [19]:
# run
nodes = get_nodes_adaptive(code_documents)
print("generated nodes:", len(nodes))
inspect_nodes(nodes)
empty_exts = [n for n in nodes if not n.metadata.get("file_ext")]
print("nodes with empty file_ext:", len(empty_exts))
if empty_exts:
    for n in empty_exts[:5]:
        print("file_name:", n.metadata.get("file_name"), "chunk_id:", n.metadata.get("chunk_id"))

generated nodes: 170

--- Sample Chunk for .py test.py
[Node Type: TextNode]
chunk_id: 0
import logging\nimport torch\nfrom os import path as osp\nimport archs\nimport data\nimport losses\nimport models\n\nfrom basicsr.data import build_dataloader, build_dataset\nfrom basicsr.models import build_model\nfrom basicsr.utils import get_env_info, get_root_logger, get_time_str, make_exp_dirs\nfrom basicsr.utils.options import dict2str, parse_options\n\n\ndef test_pipeline(root_path):\n    # parse options,...

pre_context (first 200 chars):


post_context (first 200 chars):


--- Sample Chunk for .md README.md
[Node Type: TextNode]
chunk_id: 0
# [TPAMI 2025] Diff-Retinex++: Retinex-Driven Reinforced Diffusion Model for Low-Light Image Enhancement...

pre_context (first 200 chars):


post_context (first 200 chars):
### [Paper](https://ieeexplore.ieee.org/abstract/document/10974676) | [Code](https://github.com/XunpengYi/Diff-Retinex-Plus) \n\n**Diff-Retinex++: Retinex-Driven Reinforced Diffusio

In [20]:
nodes[99].metadata

{'file_path': 'github_rag_engine/data/repos/Diff-Retinex-Plus/scripts/options.py',
 'file_name': 'options.py',
 'extension': '.py',
 'repo_path': 'github_rag_engine/data/repos/Diff-Retinex-Plus/scripts',
 'chunk_id': 3,
 'file_ext': '.py',
 'pre_context': 'parser = argparse.ArgumentParser()\n    parser.add_argument(\'-opt\', type=str, default="options/train_diff_retinex_plus_lolv1.yml", help=\'Path to option YAML file.\')\n    parser.add_argument(\'--launcher\', choices=[\'none\', \'pytorch\', \'slurm\'], default=\'none\', help=\'job launcher\')\n    parser.add_argument(\'--auto_resume\', action=\'store_true\')\n    parser.add_argument(\'--debug\', action=\'store_true\')\n    parser.add_argument(\'--local_rank\', type=int, default=0)\n    parser.add_argument(\n        \'--force_yml\', nargs=\'+\', default=None, help=\'Force to update yml files. Examples: train:ema_decay=0.999\')\n    args = parser.parse_args()\n\n    # parse yml to dict\n    with open(args.opt, mode=\'r\') as f:\n     

In [21]:
# ==========================================
# Step 8: Embeddings with Caching
# ==========================================
from llama_index.core.ingestion import IngestionPipeline, IngestionCache
from llama_index.core.storage.kvstore import SimpleKVStore
import os

In [22]:
# 1. Setup the Cache Storage
CACHE_FILE = "./github_rag_engine/cache/ingestion_cache.json"

In [23]:
os.makedirs(os.path.dirname(CACHE_FILE), exist_ok=True)
if os.path.exists(CACHE_FILE):
    print(f"📖 Loading ingestion cache from {CACHE_FILE}...")
    kv_store = SimpleKVStore.from_persist_path(CACHE_FILE)
else:
    print("✨ Creating new ingestion cache...")
    kv_store = SimpleKVStore()

✨ Creating new ingestion cache...


In [24]:
# 2. Define the Pipeline
pipeline = IngestionPipeline(
    transformations=[
        Settings.embed_model  # The embedding model is configured earlier
    ],
    cache=IngestionCache(cache=kv_store)
)

In [25]:
print(f"🚀 Generating embeddings for {len(nodes)} nodes (with caching)...")
nodes_with_embeddings = pipeline.run(nodes=nodes)
kv_store.persist(CACHE_FILE)
print(f"✅ Embeddings generated and cache saved to {CACHE_FILE}")

🚀 Generating embeddings for 170 nodes (with caching)...


2026-02-03 22:15:38,563 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-03 22:15:40,008 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


✅ Embeddings generated and cache saved to ./github_rag_engine/cache/ingestion_cache.json


In [26]:
print(f"Sample Embedding (First 5 dims): {nodes_with_embeddings[0].embedding[:5]}")

Sample Embedding (First 5 dims): [-0.014153656549751759, -0.018069013953208923, 0.06104068085551262, -0.055009569972753525, -0.01625724881887436]
